In [ ]:
import csv
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
%matplotlib inline

In [ ]:
log = pd.read_csv('data/driving_log.csv', nrows=10)
log.head()


In [ ]:
log = pd.read_csv('data/driving_log.csv', usecols=['center', 'left', 'right', 'steering'])
#log = log[log['steering']!=0].reset_index()
log.head()

In [ ]:
def preprocess(image):
    #image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)[:,:,1]
    image = image/255 - 0.5
    image = image[:,:,None]
    
    return image

In [ ]:
_ = log['steering'].hist(bins=100)

In [ ]:
log['steering'].describe()

In [ ]:
images = []
measurements = []

for i in range(len(log)):
    filename = 'data/'+log['center'][i].strip()
    
    image = mpimg.imread(filename)
    image = preprocess(image)
    measurement = log['steering'][i]
    
    images.append(image)
    images.append(cv2.flip(image, 1)[:,:,None])
    
    measurements.append(measurement)
    measurements.append(-measurement)

In [ ]:
plt.imshow(images[310].squeeze())

In [ ]:
plt.imshow(images[311].squeeze())

In [ ]:
X_train = np.array(images)
y_train = np.array(measurements)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D

In [ ]:
model = Sequential()
#model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,1)))
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160,320,1)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2) 

In [ ]:
model.predict(X_train[0][None,:,:,:])

In [ ]:
model.save('model.h5')

In [ ]:
import keras.models as km

In [ ]:
model1 = km.load_model('model.h5')

In [ ]:
model.predict(X_train[0][None,:,:,:])